In [37]:
mapp='peptide_sample_inf'
files = os.listdir(mapp)
# gets all xlsx files in cwd
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
df_inf = pd.DataFrame()
for file in files_xlsx:
    data = pd.read_excel(mapp+ '/' +file, 'peptide')
    data = data.loc[(data['#Feature'] >= 4)] # sorts out if feature<4
    data['Peptide']=data['Peptide'].str.replace('[^a-zA-Z]','')
    data = data.groupby(['Peptide','Accession'], as_index=False).sum()
    df_inf = df_inf.append(data)  # appends all data
filename='refpep.xlsx'
sheetname='Blad4'
df_ref=pd.read_excel(filename, sheet_name=sheetname)
df_ref['Peptide']=df_ref['Peptide'].str.replace('[^a-zA-Z]','')
df_ref.dropna(subset=['Peptide'], inplace=True)
df_ref

KeyError: 'Peptide'

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import xlrd
from openpyxl import load_workbook

In [36]:
result=[]
for seq1 in df_ref['Peptide']:
    for seq2 in df_inf['Peptide']:
        if seq1 in seq2 and len(seq2)<(len(seq1)+6):
            result.append((seq1,seq2))
            
result=pd.DataFrame(result)
result.columns=['ref','inf']
result.drop_duplicates('inf', inplace=True)
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = load_workbook(filename)
    result.to_excel(writer, sheetname , columns=['ref','inf'])